In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import datetime as dt
from datetime import timedelta
import yfinance as yf
import statsmodels 
import statsmodels.api as sm
from statsmodels.tsa.stattools import coint
from statsmodels.tsa.stattools import adfuller
import scipy.optimize as spop

stocks = ['JPM', 'BAC']
start_date='2015-01-01'
end_date='2023-03-15'
window = 252
t_threshold = -2.5
fee = 0.001
signal_entry = 1.2
signal_exit = 0

data = pd.DataFrame()
returns = pd.DataFrame()
i=0
for stock1p in stocks[i:len(stocks):]:
    for stock2p in stocks[i+1::]:
      stock1 = yf.Ticker(stock1)  
      price1 = 
      price2 = yf.download(stock2, start_date, end_date)   
      data[stock1] = price1['Close']
      data[stock2] = price2['Close']    
          
      stock1_rel = data[stock1]/data[stock1][0]
      stock2_rel = data[stock2]/data[stock2][0]  
      plt.plot(stock1_rel, label = stock1)
      plt.plot(stock2_rel , label = stock2) 
      plt.legend()  
      plt.show()  
      Y = np.log(data[stock1])
      X = np.log(data[stock2])
      Z = sm.add_constant(X) 
      model = sm.OLS(Y , Z)
      results = model.fit()
      results.params
      

      alpha = results.params.values[0]
      beta =  results.params.values[1]  
      errors = Y - (alpha + X * beta) 

      dftest = adfuller(errors , maxlag = 1)
      dfoutput = pd.Series(dftest[0:4] , index=["Test Stat" , "Pval" , "Lag used" , "No of Obs"])  
      critical_values = pd.Series(dftest[4].values() , index = dftest[4].keys())
      print(dfoutput)
      print(critical_values)  

      spread = errors
      zscore = (spread - np.mean(spread)) / np.std(spread)
      zscore.plot()
      plt.axhline(y = 1.2 , color = 'b')
      plt.axhline(y = -1.2 , color = 'b')
      plt.axhline(y = np.mean(spread) , color = 'r' , linestyle = '--')  
      plt.show()  

      data["short_signal"] = (zscore > signal_entry) 
      data["short_exit"] = (zscore < signal_exit) & (zscore.shift(1) > signal_exit)
      data["long_signal"] = (zscore < -signal_entry) & (zscore.shift(1) > -signal_entry)
      data["long_exit"] = (zscore > signal_exit) & (zscore.shift(1) < signal_exit)  


      backtest_res = []; indicator = 0
      spread_side = None; counter = -1
      for time,signals_stock in data.iterrows():
          counter = counter + 1
          stock2_, stock1_, short_signal, short_exit, long_signal, long_exit = signals_stock
          if spread_side == None:
              return_stock2 = 0
              return_stock1 = 0
              backtest_res.append([time, return_stock2, return_stock1, spread_side])

              if data["short_signal"][time] == True:
                  spread_side = "short"
              elif data["long_signal"][time] == True:
                  spread_side = "long"

          elif spread_side == "long" :
              return_stock2 = data[stock2][counter] / data[stock2][counter-1] - 1
              return_stock1 = data[stock1][counter] / data[stock1][counter-1] - 1
              backtest_res.append([time, return_stock2, -return_stock1, spread_side])

              if data["long_exit"][time] == True:
                  spread_side = None

          elif  spread_side == "short" :
              return_stock2 = data[stock2][counter] / data[stock2][counter-1] - 1
              return_stock1 = data[stock1][counter] / data[stock1][counter-1] - 1
              backtest_res.append([time, -return_stock2, return_stock1, spread_side])

              if data["short_exit"][time] == True:
                  spread_side = None

    
      colum = ["Date", "stock2", "stock1", "Side"]  
      backtest_pandas = pd.DataFrame(data=backtest_res, columns=colum)
      backtest_pandas["stock2 PL"] = np.cumprod(backtest_pandas["stock2"]+1)  
      backtest_pandas["stock1 PL"] = np.cumprod(backtest_pandas["stock1"]+1)
      backtest_pandas["Total PL"] = (backtest_pandas["stock1 PL"] + backtest_pandas["stock2 PL"]) / 2
      backtest_pandas.index = backtest_pandas["Date"]
      print(backtest_pandas["stock2 PL"])
      backtest_pandas[["Total PL"]].plot()
      plt.xlabel("days")
      plt.plot()  
      plt.show()
    i=i+1



import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import datetime as dt
from datetime import timedelta
import yfinance as yf
import statsmodels 
import statsmodels.api as sm
from statsmodels.tsa.stattools import coint
from statsmodels.tsa.stattools import adfuller
import scipy.optimize as spop

stocks = ['BAC', 'JPM' , 'GOOG']
start_date='2015-01-01'
end_date='2023-01-01'
window = 252
t_threshold = -2.5
fee = 0.001
signal_entry = 1.2
signal_exit = 0


returns = pd.DataFrame()
i=0
for stock1p in stocks[i:len(stocks)-1:]:
    for stock2p in stocks[i+1:len(stocks):]:
      stock1 = yf.Ticker(stock1p)  
      stock1_data = stock1.history(interval='1d' , start= start_date , end= end_date)
      stock2 = yf.Ticker(stock2p)
      stock2_data = stock2.history(interval='1d' , start= start_date , end= end_date)    
          
      stock1_rel = stock1_data["Close"]/stock1_data["Close"][0] * 100
      stock2_rel = stock2_data["Close"]/stock2_data["Close"][0] * 100
      plt.plot(stock1_rel, label = stock1p)
      plt.plot(stock2_rel , label = stock2p) 
      plt.legend()  
      plt.show()  
      Y = np.log(stock2_data["Close"])
      X = np.log(stock1_data["Close"])
      X = sm.add_constant(X) 
      model = sm.OLS(Y , X)
      results = model.fit()
      results.params
      

      alpha = results.params.values[0]
      beta =  results.params.values[1]  
      errors = Y - (alpha + X["Close"] * beta) 

      dftest = adfuller(errors , maxlag = 1)
      dfoutput = pd.Series(dftest[0:4] , index=["Test Stat" , "Pval" , "Lag used" , "No of Obs"],)  
      critical_values = pd.Series(dftest[4].values() , index = dftest[4].keys())
      print(dfoutput)
      print(critical_values)  

      spread = errors
      zscore = (spread - np.mean(spread)) / np.std(spread)
      zscore.plot()
      plt.axhline(y = 1.2 , color = 'b')
      plt.axhline(y = -1.2 , color = 'b')
      plt.axhline(y = np.mean(spread) , color = 'r' , linestyle = '--')  
      plt.show()  

      data = pd.DataFrame()
      data["stock2"] = stock2_data["Close"]
      data["stock1"] = stock1_data["Close"]  
      data["short_signal"] = (zscore > signal_entry) 
      data["short_exit"] = (zscore < signal_exit) & (zscore.shift(1) > signal_exit)
      data["long_signal"] = (zscore < -signal_entry) & (zscore.shift(1) > -signal_entry)
      data["long_exit"] = (zscore > signal_exit) & (zscore.shift(1) < signal_exit)  

      backtest_res = []; indicator = 0
      spread_side = None; counter = -1
      for time,signals_stock in data.iterrows(): 
          counter = counter + 1
          stock2_, stock1_, short_signal, short_exit, long_signal, long_exit = signals_stock
          if spread_side == None:
              return_stock2 = 0
              return_stock1 = 0
              backtest_res.append([time, return_stock2, return_stock1, spread_side])

              if data["short_signal"][time] == True:
                  spread_side = "short"
              elif data["long_signal"][time] == True:
                  spread_side = "long"

          elif spread_side == "long" :
              return_stock2 = data["stock2"][counter] / data["stock2"][counter-1] - 1
              return_stock1 = data["stock1"][counter] / data["stock1"][counter-1] - 1
              backtest_res.append([time, return_stock2, -return_stock1, spread_side])

              if data["long_exit"][time] == True:
                  spread_side = None

          elif  spread_side == "short" :
              return_stock2 = data["stock2"][counter] / data["stock2"][counter-1] - 1
              return_stock1 = data["stock1"][counter] / data["stock1"][counter-1] - 1
              backtest_res.append([time, -return_stock2, return_stock1, spread_side])

              if data["short_exit"][time] == True:
                  spread_side = None

      colum = ["Date", "stock2", "stock1", "Side"]  
      backtest_pandas = pd.DataFrame(data=backtest_res, columns=colum)
      backtest_pandas["stock2 PL"] = np.cumprod(backtest_pandas["stock2"]+1)  
      backtest_pandas["stock1 PL"] = np.cumprod(backtest_pandas["stock1"]+1)
      backtest_pandas["Total PL"] = (backtest_pandas["stock1 PL"] + backtest_pandas["stock2 PL"]) / 2
      backtest_pandas.index = backtest_pandas["Date"]
      print(backtest_pandas["Total PL"])  
      backtest_pandas["Total PL"].plot()
      plt.xlabel("dsa")  
      plt.plot() 
      plt.show()  
      print(str(stock1p)+' '+str(stock2p))  
    i=i+1


SyntaxError: invalid syntax (2764998018.py, line 28)